In [21]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

In [22]:
df = pd.read_csv('../data/bank_transactions_data.csv')

In [23]:
df.shape

(2512, 16)

In [24]:
df.head()

,TransactionID,AccountID,TransactionAmount,TransactionDate,TransactionType,Location,DeviceID,IP Address,MerchantID,Channel,CustomerAge,CustomerOccupation,TransactionDuration,LoginAttempts,AccountBalance,PreviousTransactionDate
0,TX000001,AC00128,14.09,2023-04-11 16:29:14,Debit,San Diego,D000380,162.198.218.92,M015,ATM,70,Doctor,81,1,5112.21,2024-11-04 08:08:08
1,TX000002,AC00455,376.24,2023-06-27 16:44:19,Debit,Houston,D000051,13.149.61.4,M052,ATM,68,Doctor,141,1,13758.91,2024-11-04 08:09:35
2,TX000003,AC00019,126.29,2023-07-10 18:16:08,Debit,Mesa,D000235,215.97.143.157,M009,Online,19,Student,56,1,1122.35,2024-11-04 08:07:04
3,TX000004,AC00070,184.50,2023-05-05 16:32:11,Debit,Raleigh,D000187,200.13.225.150,M002,Online,26,Student,25,1,8569.06,2024-11-04 08:09:06
4,TX000005,AC00411,13.45,2023-10-16 17:51:24,Credit,Atlanta,D000308,65.164.3.100,M091,Online,26,Student,198,1,7429.40,2024-11-04 08:06:39


In [25]:
df.describe()

,TransactionAmount,CustomerAge,TransactionDuration,LoginAttempts,AccountBalance
count,2512.000000,2512.000000,2512.000000,2512.000000,2512.000000
mean,297.593778,44.673965,119.643312,1.124602,5114.302966
std,291.946243,17.792198,69.963757,0.602662,3900.942499
min,0.260000,18.000000,10.000000,1.000000,101.250000
25%,81.885000,27.000000,63.000000,1.000000,1504.370000
50%,211.140000,45.000000,112.500000,1.000000,4735.510000
75%,414.527500,59.000000,161.000000,1.000000,7678.820000
max,1919.110000,80.000000,300.000000,5.000000,14977.990000


In [26]:
df.isnull().sum()

TransactionID              0
AccountID                  0
TransactionAmount          0
TransactionDate            0
TransactionType            0
Location                   0
DeviceID                   0
IP Address                 0
MerchantID                 0
Channel                    0
CustomerAge                0
CustomerOccupation         0
TransactionDuration        0
LoginAttempts              0
AccountBalance             0
PreviousTransactionDate    0
dtype: int64

In [27]:
df.nunique()

TransactionID              2512
AccountID                   495
TransactionAmount          2455
TransactionDate            2512
TransactionType               2
Location                     43
DeviceID                    681
IP Address                  592
MerchantID                  100
Channel                       3
CustomerAge                  63
CustomerOccupation            4
TransactionDuration         288
LoginAttempts                 5
AccountBalance             2510
PreviousTransactionDate     360
dtype: int64

In [28]:
initial_rows = df.shape[0]
df = df.drop_duplicates()

In [29]:
print(f"Initial rows: {initial_rows}, Rows after dropping duplicates: {df.shape[0]}")

Initial rows: 2512, Rows after dropping duplicates: 2512


In [30]:
for col in ['TransactionDate', 'PreviousTransactionDate']:
    df[col] = pd.to_datetime(df[col])
    print(f"Converted '{col}' to datetime")

Converted 'TransactionDate' to datetime
Converted 'PreviousTransactionDate' to datetime


In [31]:
le = LabelEncoder()

categorical_cols = ['TransactionID', 'AccountID', 'TransactionType', 'Location', 'DeviceID', 'IP Address',
                    'MerchantID', 'Channel', 'CustomerOccupation']
for col in categorical_cols:
    df[col] = le.fit_transform(df[col])
    print(f"Encoded column '{col}'")

Encoded column 'TransactionID'
Encoded column 'AccountID'
Encoded column 'TransactionType'
Encoded column 'Location'
Encoded column 'DeviceID'
Encoded column 'IP Address'
Encoded column 'MerchantID'
Encoded column 'Channel'
Encoded column 'CustomerOccupation'


In [32]:
scaler = MinMaxScaler()

num_cols = ['TransactionAmount', 'CustomerAge', 'TransactionDuration', 'LoginAttempts', 'AccountBalance']
df[num_cols] = scaler.fit_transform(df[num_cols])
print("\nNormalized numerical columns")


Normalized numerical columns


In [33]:
mask = df['PreviousTransactionDate'] > df['TransactionDate']
df.loc[mask, ['TransactionDate', 'PreviousTransactionDate']] = df.loc[mask, ['PreviousTransactionDate', 'TransactionDate']].values
print(f"Swapped TransactionDate and PreviousTransactionDate in {mask.sum()} rows")

Swapped TransactionDate and PreviousTransactionDate in 2512 rows


In [34]:
df = df.sort_values(by='TransactionDate')
df['time_diff'] = df['TransactionDate'].diff().dt.total_seconds().fillna(0)
print("Added 'time_diff' feature")

Added 'time_diff' feature


In [35]:
df['location_change'] = df.groupby('AccountID')['Location'].transform(lambda x: (x != x.shift()).astype(int))
print("Added 'location_change' feature")

Added 'location_change' feature


In [36]:
df.head()

,TransactionID,AccountID,TransactionAmount,TransactionDate,TransactionType,Location,DeviceID,IP Address,MerchantID,Channel,CustomerAge,CustomerOccupation,TransactionDuration,LoginAttempts,AccountBalance,PreviousTransactionDate,time_diff,location_change
469,469,34,0.037361,2024-11-04 08:06:23,0,29,395,188,99,2,0.661290,1,0.148276,0.0,0.616134,2023-02-13 16:39:52,0.0,1
2044,2044,237,0.098158,2024-11-04 08:06:23,1,4,436,24,82,2,0.790323,2,0.055172,0.0,0.474991,2024-01-01 17:16:23,0.0,1
1252,1252,455,0.197660,2024-11-04 08:06:23,0,16,648,25,39,1,0.370968,0,0.372414,0.0,0.838256,2023-01-03 17:14:14,0.0,1
35,35,263,0.064883,2024-11-04 08:06:23,1,39,41,479,1,1,0.500000,1,0.120690,0.0,0.380693,2023-10-16 16:32:56,0.0,1
2157,2157,153,0.107846,2024-11-04 08:06:23,1,8,586,340,27,2,0.677419,1,0.424138,0.0,0.537194,2024-01-01 16:12:32,0.0,1


In [39]:
df.to_csv('../data/clean_transactions_data.csv', index=False)

In [42]:
df.columns

Index(['TransactionID', 'AccountID', 'TransactionAmount', 'TransactionDate',
       'TransactionType', 'Location', 'DeviceID', 'IP Address', 'MerchantID',
       'Channel', 'CustomerAge', 'CustomerOccupation', 'TransactionDuration',
       'LoginAttempts', 'AccountBalance', 'PreviousTransactionDate',
       'time_diff', 'location_change'],
      dtype='object')